In [1]:
import numpy as np
from transformers import AutoTokenizer
# from transformers import GPTNeoXForCausalLM
from gpt_neox_mlkv import GPTNeoXConfig, GPTNeoXForCausalLM
# from transformers import GPTNeoXConfig, GPTNeoXForCausalLM
# check for cuda
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device:", device)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using device: cuda


In [2]:
# load model
# tokenizer = AutoTokenizer.from_pretrained('../pythia-160m-deduped_mlkv_12_12')
tokenizer = AutoTokenizer.from_pretrained('../pythia-70m-deduped')
# config = GPTNeoXConfig.from_pretrained('../pythia-160m-deduped')
# model = GPTNeoXForCausalLM.from_pretrained('../pythia-160m-deduped_mlkv_mlp_4_3')
# model = GPTNeoXForCausalLM.from_pretrained('../pythia-70m-deduped')
# tokenizer = AutoTokenizer.from_pretrained('EleutherAI/pythia-160m-deduped')
# model = GPTNeoXForCausalLM.from_pretrained('../pythia-160m-deduped')
model = GPTNeoXForCausalLM.from_pretrained('faridlazuarda/pythia-160m-mlkv-mlp-2-b12-g2-v1', revision='master')
# model = GPTNeoXForCausalLM.from_pretrained('test')
# model = GPTNeoXForCausalLM.from_pretrained('pythia-70m-fixed/checkpoint-12000')
# model = GPTNeoXForCausalLM.from_pretrained('pythia-160m-mlkv-48/checkpoint-10536')
model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 768)
    (layers): ModuleList(
      (0): GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): KeyValueMLP(
            (dense_in): Linear(in_features=768, out_features=4259, bias=True)
            (dense_out): Linear(in_features=4259, out_features=64, bias=True)
            (act): GELUActivation()
          )
          (value): KeyValueMLP(
            (dense_in): Linear(in_features=768, out_features=4259, bias=True)
            (dense_out): Linear(in_features=4259, out_features=64, bias=True)
            (act): GELUActivation()
          )
          (dense): Linear(in_features=768, out_features=768, bias=Tru

In [4]:
prompt = """The scaling of language models to tens even hundreds of billions of parameters made possible by the highly parallelizable transformer architecture has led to the rise of large language models (LLMs) as the pinnacle of natural language processing and language inference \parencite{pope2022efficiently}. Aside from achieving state-of-the-art in various benchmarks ranging from writing code to common sense and reasoning, LLMs have also been deployed massively at scale for applications such as OpenAI's ChatGPT or their API of various models \parencite{openai2023gpt4}. But serving billion-scale models to millions of customers becomes a major challenge, requiring massive compute to fulfil the needed latency constraints. Aside from a small amount reserved for computation, memory is excessively needed for two main things. Firstly the model weights, which can be large but stay fixed once deployed. The more concerning memory consumer is the key-value cache (KV cache), which contains the stored activations of attention modules at all previous time steps. This is the result of optimizing generative inference: tokens are generated one by one, auto-regressively. To avoid unnecessary re-computation of these activations, they are stored in memory to be used for generating future tokens. At small scales, KV cache can be unassuming, and memory is mostly needed for the weights. But taking OPT-175B as a large scale example, its 175 billion parameters require 325GB of memory. Yet when inferencing a sequence length of 2048 and computing a batch of 128 generations at once, the KV cache can take up to 950GB of memory \parencite{liu2023scissorhands}. In this case, the cache is three times larger than the model weights, yet even then, KV cache can still grow indefinitely in relation to the sequence length, batch size, and model dimension. TL;DR:"""
# prompt = """Wherever you are, I'll always make you smile. Wherever you are, I'm always by your side. Whatever you say, I"""
# prompt = """Whether you're in the main venue at Montevideo, at one of our several satellite locations, or participating remotely from anywhere in the world, you and your team will leverage technology and data to"""
input_ids = tokenizer.encode(prompt, return_tensors='pt')
greedy_output = model.generate(input_ids.to(device), max_length=500, return_dict_in_generate=True)
print(greedy_output)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output.sequences[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


GreedySearchDecoderOnlyOutput(sequences=tensor([[  510, 13642,   273,  3448,  3210,   281,  7114,  1014,  8307,   273,
         25866,   273,  3602,  1160,  1896,   407,   253,  4122,  7529, 12729,
         39707, 10336,   556,  3977,   281,   253,  6054,   273,  1781,  3448,
          3210,   313,  2293, 12822,    10,   347,   253,   268,  2966,  6929,
           273,  3626,  3448,  5162,   285,  3448, 17032,   393,  4904,   445,
         41766,    92,    81,  1714,   938,  1423, 20246,   314,  7165, 43308,
           432, 17170,  1375,    14,  1171,    14,   783,    14,   435,   275,
          2710, 49602, 12319,   432,  4028,  2127,   281,  1846,  3282,   285,
         14720,    13, 21708, 12822,   452,   671,   644, 18329, 48994,   387,
          4311,   323,  4893,   824,   347,  7489, 18128,   434, 24180,    40,
          5736,   390,   616,  8990,   273,  2710,  3210,   393,  4904,   445,
         41766,    92,  5758,  2284,   938,  1508,    72,   431,    21,  7165,
          12

In [ ]:
# get parameter count
print("parameter count:", sum(p.numel() for p in model.parameters() if p.requires_grad))

parameter count: 162339980


In [ ]:
import subprocess
import re
smi = subprocess.check_output('nvidia-smi')
# extract the memory usage with regex. example:" 2467MiB / 12288MiB "
mem = int(re.search(r'(\d+)MiB / (\d+)MiB', str(smi)).group(1))
print("memory usage:", mem, "MiB")

memory usage: 1762 MiB


In [ ]:
for name, param in model.base_model.named_parameters():
    print(name, param.shape)

embed_in.weight torch.Size([50304, 768])
layers.0.input_layernorm.weight torch.Size([768])
layers.0.input_layernorm.bias torch.Size([768])
layers.0.post_attention_layernorm.weight torch.Size([768])
layers.0.post_attention_layernorm.bias torch.Size([768])
layers.0.attention.query.weight torch.Size([768, 768])
layers.0.attention.query.bias torch.Size([768])
layers.0.attention.key.dense_in.weight torch.Size([4259, 768])
layers.0.attention.key.dense_in.bias torch.Size([4259])
layers.0.attention.key.dense_out.weight torch.Size([64, 4259])
layers.0.attention.key.dense_out.bias torch.Size([64])
layers.0.attention.value.dense_in.weight torch.Size([4259, 768])
layers.0.attention.value.dense_in.bias torch.Size([4259])
layers.0.attention.value.dense_out.weight torch.Size([64, 4259])
layers.0.attention.value.dense_out.bias torch.Size([64])
layers.0.attention.dense.weight torch.Size([768, 768])
layers.0.attention.dense.bias torch.Size([768])
layers.0.mlp.dense_h_to_4h.weight torch.Size([3072, 768])

In [ ]:
# inspect pytorch_model.bin and see all the keys
state_dict = torch.load('../pythia-160m-deduped/pytorch_model.bin', map_location='cpu')
for k in state_dict.keys():
    print(k)

gpt_neox.embed_in.weight
gpt_neox.layers.0.input_layernorm.weight
gpt_neox.layers.0.input_layernorm.bias
gpt_neox.layers.0.post_attention_layernorm.weight
gpt_neox.layers.0.post_attention_layernorm.bias
gpt_neox.layers.0.attention.bias
gpt_neox.layers.0.attention.masked_bias
gpt_neox.layers.0.attention.rotary_emb.inv_freq
gpt_neox.layers.0.attention.query_key_value.weight
gpt_neox.layers.0.attention.query_key_value.bias
gpt_neox.layers.0.attention.dense.weight
gpt_neox.layers.0.attention.dense.bias
gpt_neox.layers.0.mlp.dense_h_to_4h.weight
gpt_neox.layers.0.mlp.dense_h_to_4h.bias
gpt_neox.layers.0.mlp.dense_4h_to_h.weight
gpt_neox.layers.0.mlp.dense_4h_to_h.bias
gpt_neox.layers.1.input_layernorm.weight
gpt_neox.layers.1.input_layernorm.bias
gpt_neox.layers.1.post_attention_layernorm.weight
gpt_neox.layers.1.post_attention_layernorm.bias
gpt_neox.layers.1.attention.bias
gpt_neox.layers.1.attention.masked_bias
gpt_neox.layers.1.attention.rotary_emb.inv_freq
gpt_neox.layers.1.attention.q

In [ ]:
state_dict['gpt_neox.layers.11.attention.query_key_value.weight'].shape

torch.Size([2304, 768])

In [ ]:
# want to randomize one of the layers
shape = state_dict['gpt_neox.layers.11.attention.query_key_value.weight'].shape
new_weights = np.random.normal(size=shape)
state_dict['gpt_neox.layers.11.attention.query_key_value.weight'] = torch.from_numpy(new_weights).float()
# save the new weights
torch.save(state_dict, '../pythia-160m-altered/pytorch_model.bin')

RuntimeError: Parent directory ../pythia-160m-altered does not exist.

In [ ]:
# want to randomize one of the layers
shape = state_dict['gpt_neox.layers.0.mlp.dense_4h_to_h.weight'].shape
new_weights = np.random.normal(size=shape)
state_dict['gpt_neox.layers.0.mlp.dense_4h_to_h.weight'] = torch.from_numpy(new_weights).float()
# save the new weights
torch.save(state_dict, '../pythia-70m-altered/pytorch_model.bin')

In [ ]:
# want to randomize one of the layers
shape = state_dict['embed_out.weight'].shape
new_weights = np.random.normal(size=shape)
state_dict['embed_out.weight'] = torch.from_numpy(new_weights).float()
# save the new weights
torch.save(state_dict, '../pythia-70m-altered/pytorch_model.bin')

In [ ]:
# compare old and new
old_state_dict = torch.load('../pythia-160m-deduped/pytorch_model.bin', map_location='cpu')
new_state_dict = torch.load('../pythia-160m-altered/pytorch_model.bin', map_location='cpu')
for k in old_state_dict.keys():
    print(k, np.allclose(old_state_dict[k].numpy(), new_state_dict[k].numpy()))
    

gpt_neox.embed_in.weight True
gpt_neox.layers.0.input_layernorm.weight True
gpt_neox.layers.0.input_layernorm.bias True
gpt_neox.layers.0.post_attention_layernorm.weight True
gpt_neox.layers.0.post_attention_layernorm.bias True
gpt_neox.layers.0.attention.bias True
gpt_neox.layers.0.attention.masked_bias True
gpt_neox.layers.0.attention.rotary_emb.inv_freq True
gpt_neox.layers.0.attention.query_key_value.weight True
gpt_neox.layers.0.attention.query_key_value.bias True
gpt_neox.layers.0.attention.dense.weight True
gpt_neox.layers.0.attention.dense.bias True
gpt_neox.layers.0.mlp.dense_h_to_4h.weight True
gpt_neox.layers.0.mlp.dense_h_to_4h.bias True
gpt_neox.layers.0.mlp.dense_4h_to_h.weight True
gpt_neox.layers.0.mlp.dense_4h_to_h.bias True
gpt_neox.layers.1.input_layernorm.weight True
gpt_neox.layers.1.input_layernorm.bias True
gpt_neox.layers.1.post_attention_layernorm.weight True
gpt_neox.layers.1.post_attention_layernorm.bias True
gpt_neox.layers.1.attention.bias True
gpt_neox.lay

In [ ]:
# load model
model = None
tokenizer = AutoTokenizer.from_pretrained('../pythia-70m-deduped')
prompt = """The scaling of language models to tens even hundreds of billions of parameters made possible by the highly parallelizable transformer architecture has led to the rise of large language models (LLMs) as the pinnacle of natural language processing and language inference \parencite{pope2022efficiently}. Aside from achieving state-of-the-art in various benchmarks ranging from writing code to common sense and reasoning, LLMs have also been deployed massively at scale for applications such as OpenAI's ChatGPT or their API of various models \parencite{openai2023gpt4}. But serving billion-scale models to millions of customers becomes a major challenge, requiring massive compute to fulfil the needed latency constraints. Aside from a small amount reserved for computation, memory is excessively needed for two main things. Firstly the model weights, which can be large but stay fixed once deployed. The more concerning memory consumer is the key-value cache (KV cache), which contains the stored activations of attention modules at all previous time steps. This is the result of optimizing generative inference: tokens are generated one by one, auto-regressively. To avoid unnecessary re-computation of these activations, they are stored in memory to be used for generating future tokens. At small scales, KV cache can be unassuming, and memory is mostly needed for the weights. But taking OPT-175B as a large scale example, its 175 billion parameters require 325GB of memory. Yet when inferencing a sequence length of 2048 and computing a batch of 128 generations at once, the KV cache can take up to 950GB of memory \parencite{liu2023scissorhands}. In this case, the cache is three times larger than the model weights, yet even then, KV cache can still grow indefinitely in relation to the sequence length, batch size, and model dimension. TL;DR:"""
input_ids = tokenizer.encode(prompt, return_tensors='pt')
new_model = GPTNeoXForCausalLM.from_pretrained('../pythia-160m-altered')
new_model.to(device)
greedy_output = new_model.generate(input_ids.to(device), max_length=1000, temperature=0.5)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
C:\Users\Khalid Zuhri\AppData\Roaming\Python\Python311\site-packages\transformers\generation\configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
The scaling of language models to tens even hundreds of billions of parameters made possible by the highly parallelizable transformer architecture has led to the rise of large language models (LLMs) as the pinnacle of natural language processing and language inference \parencite{pope2022efficiently}. Aside from achieving state-of-the-art in various benchmarks ranging from writing code to common sense and reasoning, LLMs have also been deployed massively at scale for applications such as OpenAI's ChatGPT or their API of various models \parencite{openai2023gpt4}. But serving billion-scale models to millions of customers becomes a major challenge, requiring massive compute to fulfil the needed latency constraints. Aside from a small amount reserved for computation, memory is excessively needed for two main things. Firstly the model weights, which can be large but stay fixed once de

In [ ]:
from datasets import load_dataset
dataset = load_dataset("zaydzuhri/the_pile_tokenized_6k")
dataset = dataset['train']
# test decode tokenizer
tokenizer.decode(dataset[0]['input_ids'])

'Auburn deputy mayor Salim Mehajer is back in the spotlight after yet another run in with police.\n\nControversial Auburn deputy mayor Salim Mehajer wants an apology from police after he was stopped twice by them in less than half an hour while driving his Ferrari.\n\nMehajer, who has rarely been out of the headlines since his extravagant traffic-stopping August wedding in western Sydney, is to face court for allegedly driving an unregistered Ferrari without a licence in Punchbowl at about 10.50pm on Saturday.\n\nJust 25 minutes later he was stopped again after ignoring police instructions not to drive and was issued with a field court attendance notice, police said.\n\nThe 29-year-old said the vehicle did not show up on the police database because it was registered online and his driver\'s licence was in the car.\n\nHe said no charges have been laid.\n\n"It\'s called media attention," Mehajer told AAP on Monday via email.\n\n"I want nothing but an apology."\n\nAfter Saturday\'s incide